In [1]:
from scipy.stats import ttest_1samp, bootstrap
import numpy as np
import pandas as pd


from scipy.stats import norm, t, kstest, shapiro
import statsmodels.api as sm
from matplotlib import pyplot as plt


import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

In [2]:
df = pd.read_excel('a_b test ecom.xlsx')
df.sample(10)

,id,time,con_treat,page,converted
186393,881582,58:50.3,control,old_page,0
163777,799548,54:54.8,treatment,new_page,0
178892,878231,15:24.4,treatment,new_page,0
154514,903536,04:21.6,control,old_page,0
18218,942276,12:53.3,treatment,new_page,0
49051,825093,28:17.7,treatment,new_page,0
230046,697297,32:40.6,treatment,new_page,0
241953,631053,39:19.8,treatment,new_page,0
47604,690438,14:20.2,treatment,new_page,0
99727,786765,12:54.6,control,old_page,0


In [3]:
df.shape

(294478, 5)

In [4]:
df[df.con_treat == 'control']['page'].unique()

array(['old_page', 'new_page'], dtype=object)

In [5]:
df[(df.con_treat == 'control') & (df.page == 'new_page')]

,id,time,con_treat,page,converted
22,767017,58:15.0,control,new_page,0
240,733976,11:16.4,control,new_page,0
490,808613,44:01.3,control,new_page,0
846,637639,09:52.7,control,new_page,1
850,793580,25:33.7,control,new_page,1
...,...,...,...,...,...
293894,741581,49:03.4,control,new_page,0
293996,942612,52:28.2,control,new_page,0
294200,928506,32:10.5,control,new_page,0
294253,886135,49:20.5,control,new_page,0


In [6]:
df[(df.con_treat == 'treatment') & (df.page == 'old_page')]

,id,time,con_treat,page,converted
308,857184,34:59.8,treatment,old_page,0
327,686623,26:40.7,treatment,old_page,0
357,856078,29:30.4,treatment,old_page,0
685,666385,11:54.8,treatment,old_page,0
713,748761,47:44.4,treatment,old_page,0
...,...,...,...,...,...
293773,688144,34:50.5,treatment,old_page,1
293817,876037,15:09.0,treatment,old_page,1
293917,738357,37:55.7,treatment,old_page,0
294014,813406,25:33.2,treatment,old_page,0


In [7]:
df.id.value_counts()

805339    2
754884    2
722274    2
783176    2
898232    2
         ..
642985    1
771499    1
923606    1
712675    1
715931    1
Name: id, Length: 290584, dtype: int64

In [8]:
df[df.id==805339]

,id,time,con_treat,page,converted
90794,805339,45:15.6,treatment,old_page,0
234436,805339,00:09.0,control,old_page,0


In [9]:
index_duble = df[df.id.duplicated()].index

In [10]:
index_duble

Int64Index([  2656,   2893,   7500,   8036,  10218,  11215,  11428,  12858,
             13262,  14487,
            ...
            293996, 294014, 294200, 294252, 294253, 294308, 294309, 294328,
            294331, 294355],
           dtype='int64', length=3894)

In [11]:
clean_df = df.drop(index=index_duble)

In [12]:
clean_df.id.value_counts()

851104    1
670761    1
743992    1
945784    1
681915    1
         ..
938536    1
691858    1
642858    1
904523    1
715931    1
Name: id, Length: 290584, dtype: int64

In [13]:
n_B, n_A = clean_df.page.value_counts()
n_B, n_A

(145320, 145264)

In [14]:
m_A = clean_df[clean_df.page == 'old_page']['converted'].sum()
m_B = clean_df[clean_df.page == 'new_page']['converted'].sum()
m_A, m_B

(17489, 17256)

In [15]:
from scipy.stats import chi2_contingency

stat, p_value, _, _ = chi2_contingency([[m_A, n_A - m_A], [m_B, n_B - m_B]], correction=False)
stat, p_value

(1.8781654292993426, 0.1705428052317988)

In [16]:
A = clean_df[clean_df.page=='old_page'][['converted']].rename({'converted': 'A'}, axis=1)
B = clean_df[clean_df.page=='new_page'][['converted']].rename({'converted': 'B'}, axis=1)

In [17]:
!pip install bootstrapped

In [18]:
import bootstrapped.bootstrap as bs
import bootstrapped.compare_functions as bs_compare
import bootstrapped.stats_functions as bs_stats

In [19]:
b = bs.bootstrap_ab(A.A.values, B.B.values, stat_func=bs_stats.mean,
                    compare_func=bs_compare.difference, alpha=0.05, num_iterations=3000)
print(b.lower_bound, b.upper_bound)

-0.0006700521843055734 0.004052153082098226
